In [1]:
# Utils analysis
from utils_analysis import *

# Standard
import torch
import numpy as np
import pandas as pd
import os
import sys; sys.path.append("../")

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
%matplotlib inline
%config InlineBackend.figure_format='retina'

Making run overview, based on dir: /home/cbarkhof/code-thesis/NewsVAE/Runs
Making run overview of /home/cbarkhof/code-thesis/NewsVAE/Runs, in /home/cbarkhof/code-thesis/NewsVAE/final-analysis/Runs_run_overview.csv
Reading last checkpoint and extracting pareto dict and saving it to a pickle.
Reading all pareto dicts and calculating best checkpoint, saving it to a csv
--------------------------------------------------
Making run overview, based on dir: /home/cbarkhof/code-thesis/NewsVAE/Runs-ablation
Making run overview of /home/cbarkhof/code-thesis/NewsVAE/Runs-ablation, in /home/cbarkhof/code-thesis/NewsVAE/final-analysis/Runs-ablation_run_overview.csv
Reading last checkpoint and extracting pareto dict and saving it to a pickle.
Reading all pareto dicts and calculating best checkpoint, saving it to a csv
--------------------------------------------------
Making run overview, based on dir: /home/cbarkhof/code-thesis/NewsVAE/Runs-target-rate
Making run overview of /home/cbarkhof/code-the

In [4]:
EXP_NAME = "Runs-ablation"
VAL_BATCHES = 10

In [5]:
df = read_overview_csv(exp_name=EXP_NAME)
df

,clean_name,run_name,dataset,optimisation
0,YELP | CYC+FB-0.5 | mem,2021-05-17-17-MAY-yelp-cyclical-free-bits-0.5-...,yelp,CYC+FB-0.5
1,YELP | CYC | mem,2021-05-18-18-MAY-yelp-cyclical-only-run-11:39:28,yelp,CYC
2,YELP | FB-0.5 | mem,2021-05-19-18-MAY-yelp-free-bits-0.5-only-run-...,yelp,FB-0.5
3,YELP | MDR-0.5 | mem,2021-05-19-18-MAY-yelp-mem-mdr-0.5-run-03:39:45,yelp,MDR-0.5


In [13]:
all_sum_stats = {}
for row_i, row in df.iterrows():
    run_name = row["run_name"]
    sum_stats = get_sum_stats_run(run_name, exp_name=EXP_NAME, val_batches=VAL_BATCHES)
    for s in list(sum_stats.keys()):
        if torch.is_tensor(sum_stats[s]):
            if sum_stats[s].dim() > 1:
                del sum_stats[s]
            else:
                sum_stats[s] = sum_stats[s].mean().item()
        else:
            sum_stats[s] = np.mean(sum_stats[s])
    all_sum_stats[run_name] = sum_stats
    all_sum_stats[run_name]["clean_name"] = row["clean_name"]

In [17]:
pd.DataFrame(all_sum_stats).transpose().set_index("clean_name")

,lens_x_gen,lens,iw_ll_x_gen_p_w,iw_ll_x_gen,iw_ll_p_w,iw_ll,reconstruction_loss,exact_match,cross_entropy_per_word,kl_analytical,...,MI,mmd_loss,std_x_std,mean_x_std,std_x_mu,mean_x_mu,std_z_std,std_z_mu,log_q_z,log_q_z_mean
clean_name,,,,,,,,,,,,,,,,,,,,,
YELP | CYC+FB-0.5 | mem,59.45,58.7594,-3.46765,-204.917,-3.22819,-188.271,168.077,0.419033,2.90212,24.3785,...,13.5565,0.00359826,0.0227292,0.415748,0.746435,0.0019114,0.0137901,0.766609,-23.9178,-23.9178
YELP | CYC | mem,58.4891,58.7594,-3.0296,-176.373,-3.09249,-180.818,179.924,0.379959,3.14065,1.97291,...,10.3786,0.00209793,0.00748079,0.836212,0.248074,-0.00441818,0.0110178,0.255857,-40.9211,-40.9211
YELP | FB-0.5 | mem,59.9516,58.7594,-3.10384,-184.985,-3.07954,-179.507,163.655,0.427566,2.83027,19.4702,...,13.2666,0.00291264,0.0805526,0.495891,0.712542,-0.00548812,0.0872239,0.728416,-28.8323,-28.8323
YELP | MDR-0.5 | mem,59.0187,58.7594,-3.07535,-180.867,-3.06334,-179.022,164.998,0.422863,2.86471,16.801,...,13.5157,0.00308595,0.0414264,0.655657,0.625101,-0.0203987,0.253329,0.736689,-35.8796,-35.8796
